In [1]:
%pwd

'/workspaces/house_price_prediction/notebook'

In [2]:
import os
os.chdir('../')
%pwd

'/workspaces/house_price_prediction'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from src.constants import *
from src.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath= CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir

        )
        return data_ingestion_config

In [6]:
import os
import urllib.request as request
from src import logger
import zipfile

In [7]:
## component-Data Ingestion

class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config
    
    # Downloading the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-12-30 18:12:56,767: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-12-30 18:12:56,770: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-30 18:12:56,771: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-30 18:12:56,772: INFO: common: created directory at: artifacts]
[2024-12-30 18:12:56,772: INFO: common: created directory at: artifacts/data_ingestion]
[2024-12-30 18:12:57,374: INFO: 535005154: artifacts/data_ingestion/data.zip download! with following info: 
Access-Control-Allow-Origin: https://render.githubusercontent.com
content-disposition: attachment; filename=House-Prediction-main.zip
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
cross-origin-resource-policy: cross-origin
ETag: "8e9531bbaaa86a59fd06e8c5100cc9027d4900aeee8971203a0a1e6c5f741080"
Strict-Transport-Security: max-age=31536000
Vary: Authorization,Accept-Encoding,Origin
X-Content-Type-Options: 